In [63]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [64]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''

    return df

def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''

    return df

def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    query_result = pd.read_sql(query_statement, sql_connection)
    print(query_result)

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''
    with open('etl_project_log.txt', 'a') as f:
        f.write(f'{datetime.now()} - {message}\n')


In [65]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

### Task 1: Extracting information

In [66]:
#Extract the web page as text
html_page = requests.get(url).text

In [67]:
#Parse the html page    
data = BeautifulSoup(html_page, 'html.parser')

In [68]:
#Create an empty pandas DataFrame named df with columns as the table_attribs
df = pd.DataFrame(columns=table_attribs)

In [69]:
#Extract all 'tbody' attributes of the HTML object and then extract all the rows of the index 2 table using the 'tr' attribute.
tables = data.find_all('tbody')
rows = tables[2].find_all('tr')

In [70]:
#Check the contents of each row, having attribute ‘td’, for the following conditions.
#a. The row should not be empty.
#b. The first column should contain a hyperlink.
#c. The third column should not be '—'.

#Then store all entries matching the conditions to a dictionary with keys the same as entries of table_attribs. Append all these dictionaries one by one to the dataframe.

rows_to_add = []

for row in rows:
        # Extract all 'td' elements in the current row
    cells = row.find_all('td')

        # Check conditions: non-empty row, first column contains a hyperlink, third column is not '—'
    if len(cells) > 2 and cells[0].find('a') and cells[2].text.strip() != '—':
            
            # Extract data for each attribute
        country = cells[0].find('a').text.strip()
        gdp_usd_millions = cells[2].text.strip()

            # Store valid entries in a dictionary
        entry = {
            table_attribs[0]: country,
            table_attribs[1]: (gdp_usd_millions) if gdp_usd_millions else None
            }

            # Add the entry to the list of rows to add
        rows_to_add.append(entry)
    # Concatenate all rows to the DataFrame
df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)

In [71]:
df.head()

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"


In [72]:
df.tail()

,Country,GDP_USD_millions
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151
190,Tuvalu,65


### Task 2: Transform information

In [73]:
#Convert the contents of the 'GDP_USD_millions' column of df dataframe from currency format to floating numbers.
df['GDP_USD_millions'] = df['GDP_USD_millions'].str.replace(',', '').str.replace('$', '').astype(float)

In [74]:
#Divide all these values by 1000 and round it to 2 decimal places
df['GDP_USD_millions'] = (df['GDP_USD_millions'] / 1000).round(2)

In [75]:
#Modify the name of the column from 'GDP_USD_millions' to 'GDP_USD_billions'
df.rename(columns={'GDP_USD_millions': 'GDP_USD_billions'}, inplace=True)

In [76]:
df.head()

,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88


### Task 3: Loading information

In [77]:
# save the transformed dataframe to a CSV file. For this, pass the dataframe df and the CSV file path to the function load_to_csv() and add the required statements there.
load_to_csv(df, csv_path)

In [78]:
# save the transformed dataframe as a table in the database. This needs to be implemented in the function load_to_db(), which accepts the dataframe df, the connection object to the SQL database conn, and the table name variable table_name to be used.
sql_connection = sqlite3.connect(db_name)
load_to_db(df, sql_connection, table_name)

### Task 4: Querying the database table

In [ ]:
query_statement = 'SELECT * FROM Countries_by_GDP'
run_query(query_statement, sql_connection)

### Task 5: Logging progress

In [ ]:
#Logging needs to be done using the log_progress() function.
#This function will be called multiple times throughout the execution of this code
#and will be asked to add a log entry in a .txt file, etl_project_log.txt.
#The entry is supposed to be in the following format:'<Time_stamp> : <message_text>'
#Here, message text is passed to the function as an argument. Each entry must be in a separate line.
log_progress('Data extraction completed successfully.')